# 🚀 GPU-Accelerated G1 Training with JAX/MJX on Google Colab

**Train Unitree G1 humanoid robot 10-20x faster using JAX and MuJoCo MJX!**

## What You'll Get
- ⚡ **10-20x faster training** than CPU/PyTorch version
- 🎮 **1024-1536 parallel environments** on Colab T4 GPU
- ⏱️ **Train in ~1-2 hours** instead of 13+ hours
- 💾 **Automatic checkpointing** every 50 iterations
- 📊 **TensorBoard monitoring** in real-time

## Prerequisites
- Google account (for Colab)
- **GPU runtime enabled** (Runtime > Change runtime type > T4 GPU)

## Estimated Time
- Setup: ~5 minutes
- Training (1000 iterations): ~1.5 hours on T4
- Can stop anytime and resume from checkpoint

---

## Step 1: Verify GPU and Setup Environment

**IMPORTANT:** Make sure you have **GPU runtime** enabled:
1. Go to `Runtime` → `Change runtime type`
2. Select `T4 GPU` under Hardware accelerator
3. Click `Save`

In [ ]:
# Check GPU availability
!nvidia-smi

import os
os.environ['JAX_DEFAULT_MATMUL_PRECISION'] = 'highest'  # For stability on T4

print("\n✓ GPU detected! Proceeding with installation...")

## Step 2: Install JAX with CUDA Support

In [ ]:
# Install JAX with CUDA 12 (compatible with Colab T4)
!pip install --upgrade "jax[cuda12]>=0.4.23" -q

# Verify JAX installation
import jax
print(f"JAX version: {jax.__version__}")
print(f"JAX devices: {jax.devices()}")
print(f"JAX backend: {jax.default_backend()}")

if 'gpu' in str(jax.devices()[0]).lower():
    print("\n✓ JAX GPU successfully configured!")
else:
    print("\n⚠ WARNING: GPU not detected by JAX. Check runtime settings.")

## Step 3: Clone Repository and Install Dependencies

In [ ]:
# Clone repository
!git clone https://github.com/julienokumu/unitree_rl_mugym.git
%cd unitree_rl_mugym

# Install JAX/MJX dependencies
!pip install -e .[jax_gpu] -q

print("\n✓ Installation complete!")

## Step 4: Test Setup

In [ ]:
# Run setup test
!python test_jax_setup.py

## Step 5: Configure Training Parameters

**T4 GPU Recommendations:**
- `num_envs`: 1024-1536 (T4 has 16GB memory)
- `num_timesteps`: 50M-100M
- Expected speed: ~1000 iterations in 1.5 hours

In [ ]:
# Training configuration
ROBOT = "g1"
NUM_ENVS = 1024  # Optimized for T4 GPU (16GB)
NUM_TIMESTEPS = 50_000_000  # ~50M steps
MAX_ITERATIONS = 1000  # Will run for ~1.5 hours
SAVE_INTERVAL = 50  # Save every 50 iterations (~4.5 minutes)
LEARNING_RATE = 3e-4
EXPERIMENT_NAME = "g1_jax_colab_t4"

print(f"Configuration:")
print(f"  Robot: {ROBOT}")
print(f"  Parallel Envs: {NUM_ENVS}")
print(f"  Max Iterations: {MAX_ITERATIONS}")
print(f"  Expected Time: ~1.5 hours")
print(f"  Checkpoint Interval: {SAVE_INTERVAL} iterations")

## Step 6: Start Training

**Note:** This cell will run for ~1.5 hours. You can:
- Stop it anytime (checkpoints saved every 50 iterations)
- Monitor progress in TensorBoard (next cell)
- Download checkpoints periodically

In [ ]:
# Start training
!python legged_gym/scripts/train_jax_ppo.py \
  --robot {ROBOT} \
  --num_envs {NUM_ENVS} \
  --backend mjx \
  --num_timesteps {NUM_TIMESTEPS} \
  --learning_rate {LEARNING_RATE} \
  --experiment_name {EXPERIMENT_NAME} \
  --checkpoint_interval {SAVE_INTERVAL} \
  --log_interval 10

## Step 7: Monitor Training with TensorBoard

Run this in a separate cell while training:

In [ ]:
# Load TensorBoard
%load_ext tensorboard

# Find log directory
import glob
log_dirs = glob.glob('logs/g1_jax/*')
if log_dirs:
    latest_log = max(log_dirs, key=os.path.getctime)
    print(f"Monitoring: {latest_log}")
    %tensorboard --logdir {latest_log}
else:
    print("No log directories found yet. Start training first.")

## Step 8: Download Trained Models

Download checkpoints to visualize locally with MuJoCo:

In [ ]:
# List available checkpoints
import glob
import os

checkpoint_dirs = glob.glob('logs/g1_jax/*')
if checkpoint_dirs:
    latest_run = max(checkpoint_dirs, key=os.path.getctime)
    checkpoints = glob.glob(f'{latest_run}/*.pkl') + glob.glob(f'{latest_run}/*.pt')
    
    print(f"Found {len(checkpoints)} checkpoints in {latest_run}")
    for ckpt in sorted(checkpoints)[-5:]:  # Show last 5
        print(f"  - {os.path.basename(ckpt)}")
else:
    print("No checkpoints found. Train first.")

In [ ]:
# Download checkpoint (change checkpoint name as needed)
from google.colab import files

# Get latest checkpoint
checkpoint_dirs = glob.glob('logs/g1_jax/*')
if checkpoint_dirs:
    latest_run = max(checkpoint_dirs, key=os.path.getctime)
    checkpoints = glob.glob(f'{latest_run}/*.pkl') + glob.glob(f'{latest_run}/*.pt')
    
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print(f"Downloading: {latest_checkpoint}")
        files.download(latest_checkpoint)
        
        # Also download config
        config_path = f'{latest_run}/config.json'
        if os.path.exists(config_path):
            files.download(config_path)
    else:
        print("No checkpoints available yet")
else:
    print("No training runs found")

## Step 9: Compress and Download All Checkpoints

In [ ]:
# Compress entire training run for download
import shutil

checkpoint_dirs = glob.glob('logs/g1_jax/*')
if checkpoint_dirs:
    latest_run = max(checkpoint_dirs, key=os.path.getctime)
    run_name = os.path.basename(latest_run)
    
    # Create zip archive
    archive_name = f"{run_name}_checkpoints"
    shutil.make_archive(archive_name, 'zip', latest_run)
    
    print(f"Compressed {latest_run}")
    print(f"Archive size: {os.path.getsize(archive_name + '.zip') / 1024 / 1024:.1f} MB")
    
    # Download
    files.download(archive_name + '.zip')
else:
    print("No training runs found")

## 🎬 Visualize Locally

After downloading checkpoints, visualize on your local machine:

```bash
# Install MuJoCo locally
pip install mujoco==3.2.3 torch pyyaml

# Visualize trained policy
python deploy/deploy_mujoco/deploy_mujoco.py g1.yaml \
    --policy /path/to/downloaded/checkpoint.pkl
```

---

## 📊 Performance Comparison

| Method | Num Envs | Time (1000 iter) | Speedup |
|--------|----------|------------------|----------|
| PyTorch CPU | 256 | ~13 hours | 1x |
| JAX/MJX T4 | 1024 | ~1.5 hours | **8.7x** |

## 💡 Tips

1. **Session Management**: Colab disconnects after ~12 hours
   - Train in shorter sessions (1-2 hours)
   - Download checkpoints frequently
   - Resume from checkpoint in next session

2. **Memory Issues**: If OOM errors occur:
   - Reduce `NUM_ENVS` to 768 or 512
   - Reduce network size in training script

3. **Faster Training**: 
   - Use Colab Pro for A100 (40GB): ~4096 envs, 4x faster
   - Or rent GPU from vast.ai, runpod.io

## 🐛 Troubleshooting

**"No GPU found"**
- Check Runtime > Change runtime type > T4 GPU
- Restart runtime and run again

**"Out of Memory"**
- Reduce `NUM_ENVS` to 768
- Restart runtime to clear memory

**"Training very slow"**
- First iteration takes 30-60s (JIT compilation)
- Subsequent iterations should be fast (~5-10s)
- If still slow, verify GPU is being used

## 📚 Resources

- [Full Documentation](../docs/JAX_MJX_TRAINING.md)
- [Quickstart Guide](../QUICKSTART_JAX.md)
- [Repository](https://github.com/julienokumu/unitree_rl_mugym)

---

**Happy Training! 🤖**